<a href="https://colab.research.google.com/github/zzunebye/Capstone-code-data/blob/main/_RumorEval_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

/content
Mounted at ./MyDrive


In [7]:
cd MyDrive/MyDrive/Capstone/code_data

[Errno 2] No such file or directory: 'MyDrive/MyDrive/Capstone/code_data'
/content/MyDrive/MyDrive/Capstone/code_data


In [3]:
import pandas as pd
import numpy as np

from glob2 import glob
import json

# FUNCTION TO FECTH TEXT FROM JSON DATA

In [11]:
def fetchRawText(path, events, tweetType):
    jsons = []
    for i, event in enumerate(events):
        jsons.append(glob('%s/%s/**/%s/*.json' % (path, event,tweetType)))
    for i,d in enumerate(jsons): print("%s's length is %d" %(events[i], len(d)))

    targets = []
    features = []
    for index, dataset in enumerate(jsons):
        targetEvent = []
        dataEvent = []
        count = 0  # help var
        for jsonFile in dataset:
            count += 1
            if jsonFile.find("non-rumours") == -1:
                targetEvent.append(1)
            else:
                targetEvent.append(0)

            with open(jsonFile, 'r') as f:
                for l in f.readlines():
                    if not l.strip():  # skip empty lines
                        continue
                    # print(jsonFile)
                    json_data = json.loads(l)
                    # print (json_data,"\n\n")
                    dataEvent.append(json_data)
        print(index, events[index], len(targetEvent), len(dataEvent))
        targets.append(targetEvent)
        features.append(dataEvent)

    # print("\nNumber of Events:", len(targets))
    # print("Number of tweets in the first event:", len(targets[0]))

    # targets은 targetEvent들을 리스트에 담은 것
    target_list = []
    for event in targets:
        for elem in event:
            target_list.append(elem)
    target = pd.DataFrame(target_list, columns=["target"])

    extracted_features = []

    extracted = []

    for obj_list in features:
        extracted_event = []
        for obj in obj_list:
            output_f = dict()
            output_f['text'] = obj['text']
            extracted_event.append(output_f)
        extracted_features.append(extracted_event)

    extracted_df = []
    for i, data in enumerate(extracted_features):
        temp = pd.DataFrame(data)
        temp["Event"] = events[i]
        extracted_df.append(pd.DataFrame(temp))

    final = pd.concat(extracted_df, ignore_index=True)
    final = pd.concat([final, target], axis=1)
    return final
    

# PHEME (5 Events)

In [75]:
path = "../pheme-rnr-dataset"
events = ['charliehebdo', 'ferguson',
          'germanwings-crash', 'ottawashooting', 'sydneysiege']
tweetType = 'source-tweet'
jsons = []

In [76]:
final = fetchRawText(path, events, tweetType)

charliehebdo's length is 2079
ferguson's length is 1143
germanwings-crash's length is 469
ottawashooting's length is 890
sydneysiege's length is 1221
0 charliehebdo 2079 2079
1 ferguson 1143 1143
2 germanwings-crash 469 469
3 ottawashooting 890 890
4 sydneysiege 1221 1221


In [ ]:
final.head()

In [38]:
# final.to_csv('./data/_PHEME_text.csv', index = False)
# target.to_csv('./data/_PHEME_target.csv', index = False)
# # d = pd.read_csv("./data/_PHEME_text.csv")

# PHEME (Additional)

## Code

In [12]:
with open('../PHEME/all-rnr-annotated-threads/ebola-essien/rumours/521310417696858112/source-tweets/521310417696858112.json','r') as f:
    s = f.read()
    s = s.replace('\t','')
    s = s.replace('\n','')
    s = s.replace(',}','}')
    s = s.replace(',]',']')
    data = json.loads(s)

In [13]:
path = "../PHEME/all-rnr-annotated-threads"
events = ['ebola-essien', 'prince-toronto', 'putinmissing']
tweetType = 'source-tweets'
jsons = []
final = fetchRawText(path,events,tweetType)
final.head()


ebola-essien's length is 14
prince-toronto's length is 233
putinmissing's length is 238
0 ebola-essien 14 14
1 prince-toronto 233 233
2 putinmissing 238 238


,text,Event,target
0,Micheal Essien denying the Ebola rumours like ...,ebola-essien,1
1,No truth in internet rumours that I have contr...,ebola-essien,1
2,Essien and his lawyers are considering to file...,ebola-essien,1
3,Good news: The rumours that Michael Essien has...,ebola-essien,1
4,Milan have stated that the reports about Essie...,ebola-essien,1


In [14]:
final.to_csv('./data/_PHEMEext_text.csv', index = False)
# target.to_csv('./data/_PHEMEext_target.csv', index = False)
# # d = pd.read_csv("./data/_PHEME_text.csv")

## Process

In [42]:
path = "../PHEME/all-rnr-annotated-threads"

putinmissing_jsons = glob('../pheme-rumour-scheme-dataset/threads/en/putinmissing/**/source-tweets/*.json')
germanwing_scrash_jsons = glob('../pheme-rnr-dataset/germanwings-crash/**/source-tweet/*.json')

events = ['ebola-essien', 'prince-toronto', 'putinmissing']
jsons = []

for i, event in enumerate(events):
    # print('%s/%s/*/*/source-tweets/*.json' % (path, event))
    jsons.append(glob('%s/%s/**/**/source-tweets/*.json' % (path, event)))
for i,d in enumerate(jsons): print("%s's length is %d" %(events[i], len(d)))

ebola-essien's length is 28
prince-toronto's length is 466
putinmissing's length is 476


In [43]:
targets = []
features = []
for index, dataset in enumerate(jsons):
    targetEvent = []
    dataEvent = []
    count = 0  # help var
    for jsonFile in dataset:
        count += 1
        if jsonFile.find("non-rumours") == -1:
            targetEvent.append(1)
        else:
            targetEvent.append(0)

        with open(jsonFile, 'r') as f:
            for l in f.readlines():
                if not l.strip():  # skip empty lines
                    continue
                json_data = json.loads(l)
                # print (json_data,"\n\n")
                dataEvent.append(json_data)
    print(index, events[index], len(targetEvent), len(dataEvent))
    targets.append(targetEvent)
    features.append(dataEvent)

print("\nNumber of Events:", len(targets))
print("Number of tweets in the first event:", len(targets[0]))

0 ebola-essien 28 28
1 prince-toronto 466 466
2 putinmissing 476 476

Number of Events: 3
Number of tweets in the first event: 28


In [7]:
# targets은 targetEvent들을 리스트에 담은 것
target_list = []
for event in targets:
    for elem in event:
        target_list.append(elem)
target = pd.DataFrame(target_list, columns=["target"])

In [8]:
extracted_features = []

extracted = []

for obj_list in features:
    extracted_event = []
    for obj in obj_list:
        output_f = dict()
        output_f['text'] = obj['text']
        extracted_event.append(output_f)
    extracted_features.append(extracted_event)

In [9]:
extracted_df = []
for i, data in enumerate(extracted_features):
    temp = pd.DataFrame(data)
    temp["Event"] = events[i]
    extracted_df.append(pd.DataFrame(temp))

In [11]:
final = pd.concat(extracted_df, ignore_index=True)
final = pd.concat([final, target], axis=1)
final.head()

,Event,target
